# Load Libraries and Resources

In [1]:
library('readr')
library('blabr')
library('plyr')
library('dplyr')
library('ggplot2')
library('rjson')
library('parallel')
library("readxl")
library('dotwhisker')
library('broom')
library('lme4')
library('irr')
library('tuneR')
library('ggExtra')
library('brms')
library('ggstance')
source('PLEARN_analysis_helper.R')
source('texvars.R')


Attaching package: ‘dplyr’


The following objects are masked from ‘package:plyr’:

    arrange, count, desc, failwith, id, mutate, rename, summarise,
    summarize


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union


Loading required package: Matrix

Loading required package: lpSolve

Loading required package: Rcpp

Loading 'brms' package (version 2.17.0). Useful instructions
can be found by typing help('brms'). A more detailed introduction
to the package is available through vignette('brms_overview').


Attaching package: ‘brms’


The following object is masked from ‘package:lme4’:

    ngrps


The following object is masked from ‘package:stats’:

    ar



Attaching package: ‘ggstance’


The following objects are masked from ‘package:ggplot2’:

    geom_errorbarh, GeomErrorbarh




# Prepare Subject Data

In [2]:
subject_info = read.csv('subject_info.csv', stringsAsFactors=F) #Subject infor without PII
subject_info$participant_name = sapply(strsplit(subject_info$filename,'_' ), function(x){x[1]})

# children without eyetracking files
for (participant in c('pl04','pl05','pl35','pl58')){
    subject_info[subject_info$id == participant,'participant_name'] = participant
}  

In [3]:
# add the lookit participants
lookit_subject_info = read.csv('lookit_subject_info.csv', stringsAsFactors=F)
lookit_subject_info$filename = lookit_subject_info$id
lookit_subject_info$participant_name = lookit_subject_info$id
lookit_subject_info$type = 'child'

# need to merge in the age in months
child_ages = read.csv('csv/lookit_child_ages.csv')[,c('id', 'age_in_days')]
lookit_subject_info = merge(lookit_subject_info,child_ages[,c('id','age_in_days')])

In [4]:
subject_info = rbind.fill(subject_info, lookit_subject_info)

subject_info$age_in_months = subject_info$age_in_days / 30.5

subject_info$ran_participant = subject_info$expt_version %in% c('redblue', 
    'scene', 'agreement','agreement-lookit') &
(subject_info$type == 'adult' | (
    subject_info$age_in_months > 24 & 
    subject_info$age_in_months < 36 &
    subject_info$type == 'child'
))

In [5]:
# exclude any child participants that are outside of the age range
subject_info = subset(subject_info, 
    type == 'adult' | (
    age_in_months >= 24 & 
    age_in_months <= 36 &
    type == 'child')
)

# Save the R objects for loading in later notebooks

In [6]:
saveRDS(subject_info, file='intermediate_data/subject_info.RData')